<a href="https://colab.research.google.com/github/pedrovmjm/PLN/blob/main/Atividade%204/2023_Q3_PLN_ATIVIDADE_PR%C3%81TICA_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2023.Q3]**
Prof. Alexandre Donizeti Alves

### **ATIVIDADE PRÁTICA 04 [Uso da API da OpenAI com técnicas de PLN]**


A **ATIVIDADE PRÁTICA 04** deve ser feita utilizando o **Google Colab** com uma conta sua vinculada ao Gmail. O link do seu notebook, armazenado no Google Drive, além do link de um repositório no GitHub e os principais resultados da atividade, devem ser enviados usando o seguinte formulário:

> https://forms.gle/GzwCq3R7ExtE9g9a8


**IMPORTANTE**: A submissão deve ser feita até o dia 20/11 (segunda-feira) APENAS POR UM INTEGRANTE DA EQUIPE, até às 23h59. Por favor, lembre-se de dar permissão de ACESSO IRRESTRITO para o professor da disciplina de PLN.

### **EQUIPE**

---

**POR FAVOR, PREENCHER OS INTEGRANDES DA SUA EQUIPE:**


**Integrante 01:**

`Por favor, informe o seu nome completo e RA:`
Pedro Victor Marcelino Jordão Motta RA: 11201921599

**Integrante 02:**

`Por favor, informe o seu nome completo e RA:`
Luccas Vinicius de Faveri Tortorelli Cardoso 11201920991

### **LIVRO**
---

`Processamento de Linguagem Natural - Conceitos, Técnicas e Aplicações em Português.`

>

Disponível gratuitamente em:
  
  > https://brasileiraspln.com/livro-pln/1a-edicao/.


**POR FAVOR, PREENCHER OS CAPITULOS SELECIONADOS PARA A SUA EQUIPE:**

`Primeiro capítulo: ` 8

`Segundo capítulo:` 18



### **DESCRIÇÃO**
---

Implementar um `notebook` no `Google Colab` que faça uso da **API da OpenAI** aplicando, no mínimo, 3 técnicas de PLN. As técnicas devem ser aplicadas nos 2 (DOIS) capítulos do livro **Processamento de Linguagem Natural - Conceitos, Técnicas e Aplicações em Português**.

>

**RESTRIÇÃO**: É obrigatório usar o *endpoint* "*`Chat Completions`*".

>

As seguintes técnicas de PLN podem ser usadas:

*   Correção Gramatical
*   Classificação de Textos
*   Análise de Sentimentos
*   Detecção de Emoções
*   Extração de Palavras-chave
*   Tradução de Textos
*   Sumarização de Textos
*   **Similaridade de Textos**
*   **Reconhecimento de Entidades Nomeadas**
*   **Sistemas de Perguntas e Respostas**

>

Os capítulos devem ser os mesmos selecionados na **ATIVIDADE PRÁTICA 02**. Para consultar os capítulos, considere a seguinte planilha:

>

> https://docs.google.com/spreadsheets/d/1ZutzQ3v1OJgsgzCvCwxXlRIQ3ChXNlHNvB63JQvYsbo/edit?usp=sharing

>
>

**IMPORTANTE:** É obrigatório usar o e-mail da UFABC. Não é permitido alterar os capítulos já selecionados.



### **CRITÉRIOS DE AVALIAÇÃO**
---


Serão considerados como critérios de avaliação as técnicas usadas e a criatividade envolvida na aplicação das mesmas.




### **IMPLEMENTAÇÃO**
---

#### **Instalação de Libs**

In [ ]:
# Instalação da biblioteca Faiss para operações eficientes de busca e análise em conjuntos grandes de vetores.
# A versão '-cpu' indica que a instalação é para CPUs, apropriada quando não se utiliza aceleração de hardware especializada, como GPUs.
!pip install faiss-cpu


In [ ]:
# Instalação da versão específica 0.28.1 da biblioteca OpenAI.
# Fixar a versão pode ser útil para manter consistência em projetos e evitar problemas de compatibilidade.
!pip install openai==0.28.1

In [ ]:
# Instalação da versão específica 0.0.329 da biblioteca Langchain.
# Fixar a versão pode ser útil para manter consistência em projetos e evitar problemas de compatibilidade.
!pip install langchain==0.0.329


In [ ]:
# Instalação da biblioteca tiktoken, que fornece uma contagem de tokens para textos no formato do OpenAI GPT.
!pip install tiktoken

#### **Importação de todas as Libs para a tarefa**

In [ ]:
# Importação de bibliotecas necessárias para o script.
from bs4 import BeautifulSoup  # Para realizar o parsing de HTML.
import requests  # Para fazer requisições HTTP.
import re  # Para manipulação de expressões regulares.
import time  # Para manipulação de tempo.
import json  # Para manipulação de dados em formato JSON.
import openai  # Biblioteca OpenAI para tarefas de linguagem natural.
from google.colab import files  # Para manipulação de arquivos no ambiente Google Colab.
import textwrap  # Para facilitar a formatação de texto.
from langchain.text_splitter import CharacterTextSplitter  # Módulo para dividir texto em caracteres.
from langchain.embeddings.openai import OpenAIEmbeddings  # Módulo para incorporar vetores utilizando OpenAI.
from langchain.vectorstores import FAISS  # Módulo para armazenamento eficiente de vetores utilizando FAISS.
from langchain.llms import OpenAI  # Módulo para modelos de linguagem utilizando OpenAI.
import httpx  # Cliente HTTP assíncrono.

#### **Definindo a chave da API**

In [ ]:
# fazer upload do arquivo de texto
upload_arquivo = files.upload()

# obter o nome do arquivo
nome_arquivo = list(upload_arquivo.keys())[0]

# ler o conteúdo do arquivo
with open(nome_arquivo, 'r') as file:
   chave_api = file.read()

# definir a chave da API
openai.api_key = chave_api

Saving apiOpenai.txt to apiOpenai (2).txt


#### **Funções para a tarefa de resumir**

In [ ]:
# Definição da função para obter texto de um item.
def getText(item):
    # Mensagem de início da captura do capítulo.
    print(f"Iniciando a captura do capítulo: {item[0]}")

    # Faz uma requisição HTTP para a URL do item e obtém o conteúdo HTML.
    response = requests.get(item[1])

    # Utiliza BeautifulSoup para fazer o parsing do conteúdo HTML.
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontra o elemento principal no HTML com a classe 'content' e o id 'quarto-document-content'.
    main_element = soup.find('main', {'class': 'content', 'id': 'quarto-document-content'})

    # Verifica se o elemento principal foi encontrado.
    if main_element:
        # Obtém o texto do elemento principal em minúsculas e sem normalização.
        texto = main_element.get_text().lower()
        textoOffNormalize = main_element.get_text()

    # Mensagem indicando sucesso na conexão e coleta do texto.
    print("Conexão e coleta do texto com sucesso")

    # Cria um dicionário com os dados do capítulo.
    json_data = {
        "Capitulo": item[0],
        "texto": texto,
        "textoOffNormalize": textoOffNormalize,
    }

    # Retorna o dicionário com os dados do capítulo.
    return json_data


In [ ]:
# Definição da função para quebrar um texto em chunks.
def quebraTexto(text):
    # Tamanho máximo de cada chunk (em caracteres).
    tamanho_chunk = 16384
    chunks = []  # Lista para armazenar os chunks resultantes.
    chunk_atual = ""  # String para construir o chunk atual.

    # Itera sobre as frases do texto.
    for sentence in text.split("."):
        # Verifica se adicionar a frase atual ao chunk não excede o tamanho máximo.
        if len(chunk_atual) + len(sentence) < tamanho_chunk:
            chunk_atual += sentence + "."  # Adiciona a frase ao chunk atual.
        else:
            chunks.append(chunk_atual.strip())  # Adiciona o chunk atual à lista.
            chunk_atual = sentence + "."  # Inicia um novo chunk com a frase atual.

    # Verifica se há algum chunk não processado ao final.
    if chunk_atual:
        chunks.append(chunk_atual.strip())

    # Retorna a lista de chunks.
    return chunks


In [ ]:
# Definição da função para resumir um trecho de texto utilizando a API da OpenAI.
def resumir_trecho(texto, indice, max_retries=3, delay_between_retries=5):
    # Loop para realizar tentativas de resumo.
    for tentativa in range(max_retries + 1):
        chunk = texto  # Inicializa o chunk com o texto fornecido.

        # Lista de mensagens a serem enviadas para a API da OpenAI.
        mensagens = []
        # Mensagem de sistema explicando o propósito da IA no resumo de texto.
        mensagens.append({'role': 'system', 'content': "Você é uma inteligência artificial que irá resumir pedaços de texto de um livro sobre Processamento de Linguagem Natural, portanto, você deve capturar explicações e pontos importantes para o entendimento e mantendo uma linearidade no resumo."})
        # Mensagem do usuário contendo o chunk de texto a ser resumido.
        mensagens.append({'role': 'user', 'content': chunk})
        print(mensagens)

        # Chamada à API da OpenAI para obter um resumo do texto.
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k-0613",
            messages=mensagens,
            max_tokens=2048,
            temperature=0.4,
        )

        # Verifica se a resposta da API contém escolhas válidas.
        if "choices" in response and response["choices"]:
            summary = response["choices"][0]["message"]["content"]
            return summary  # Retorna o resumo obtido.
        else:
            # Caso a tentativa não seja bem-sucedida, imprime mensagem de falha.
            print(f"Falha na tentativa {tentativa + 1}.")
            # Verifica se há mais tentativas disponíveis.
            if tentativa < max_retries:
                print(f"Tentando novamente em {delay_between_retries} segundos...")
                time.sleep(delay_between_retries)  # Aguarda antes de tentar novamente.
            else:
                print("Número máximo de tentativas atingido. Abortando.")
                return None  # Retorna None caso todas as tentativas falhem.


In [ ]:
# Definição da função para visualizar resumos de trechos.
def visualizar_resumo(tuplaResumo):
    largura_maxima = 200  # Largura máxima para formatação do texto.

    # Itera sobre os resumos na tupla.
    for i, value in enumerate(tuplaResumo):
        # Quebra o texto do resumo em linhas de acordo com a largura máxima.
        texto_quebrado = textwrap.wrap(value[1], largura_maxima)

        # Imprime o número do trecho e o resumo formatado.
        print("Resumo do trecho", value[0], ":")
        for linha in texto_quebrado:
            print(linha)
        print('\n')  # Adiciona uma linha em branco entre os resumos.

#### **Resgate do texto do capítulo 8 e 18**

In [ ]:
# Lista de URLs a serem processadas.
urls = [
    (8, "https://brasileiraspln.com/livro-pln/1a-edicao/parte5/cap8/cap8.html"),
    (18, "https://brasileiraspln.com/livro-pln/1a-edicao/parte8/cap18/cap18.html")
]

# Lista para armazenar os resultados da função getText para cada URL.
texto = []

# Itera sobre as URLs e obtém o texto de cada capítulo.
for i in urls:
    texto.append(getText(i))


Iniciando o captura do capitulo:8
Conexão e coleta do texto com sucesso
Iniciando o captura do capitulo:18
Conexão e coleta do texto com sucesso


#### **Aplicação da Tarefa de Resumo de texto**

In [ ]:
# Obtendo os chunks do texto do capítulo 8 usando a função quebraTexto.
chunks_8 = quebraTexto(texto[0]['textoOffNormalize'])

In [ ]:
# Obtendo os chunks do texto do capítulo 18 usando a função quebraTexto.
chunks_18 = quebraTexto(texto[1]['textoOffNormalize'])

In [ ]:
# Lista para armazenar os resumos do Capítulo 8.
resumo_Capitulo_8 = []

# Itera sobre os chunks do Capítulo 8 e obtém os resumos.
for i, value in enumerate(chunks_8):
    resumo = resumir_trecho(value, i)

    # Verifica se o resumo foi bem-sucedido.
    if resumo is not None:
        print(f"\n\nResumo do trecho {i} foi bem-sucedido. Segue o resumo: {resumo}\n\n")
        resumo_Capitulo_8.append((i, resumo))
    else:
        print(f"Falha no resumo do trecho {i}. Você pode tentar novamente ou lidar com o contexto de falha.")

[{'role': 'system', 'content': 'Você é uma inteligência artificial que irá resumir pedaços de texto de um livro sobre Processamento de Linguagem Natural, portanto, você deve capturar explicações e pontos importantes para o entendimento e mantendo uma linearidade no resumo.'}, {'role': 'user', 'content': '8\xa0 E o significado?\n\n\n\n\n\nCláudia Freitas \n\n\n\nPublicado em:\n\n26/09/2023\n\n\n\n\nPDF\n\nSemântica lida com o sentido do que é comunicado por meio da linguagem (em oposição ao que é comunicado por imagens ou sons não verbais, por exemplo). Assim, a semântica estuda o significado de palavras e frases. Mas a simplicidade relativa ao que é semântica acaba aí. Nos estudos linguísticos, a semântica é conhecida como “um domínio de investigação de limites movediços” e para o qual não há jargões bem estabelecidos (Ilari; Geraldi, 1985, p. 6). A questão “o que é o significado de uma palavra?” (e também o de uma frase) é um dos problemas nucleares da investigação semântica, e sua re

In [ ]:
# Lista para armazenar os resumos do Capítulo 18.
resumo_Capitulo_18 = []

# Itera sobre os chunks do Capítulo 18 e obtém os resumos.
for i, value in enumerate(chunks_18):
    resumo = resumir_trecho(value, i)

    # Verifica se o resumo foi bem-sucedido.
    if resumo is not None:
        print(f"\n\nResumo do trecho {i} foi bem-sucedido. Segue o resumo: {resumo}\n\n")
        resumo_Capitulo_18.append((i, resumo))
    else:
        print(f"Falha no resumo do trecho {i}. Você pode tentar novamente ou lidar com o contexto de falha.")

[{'role': 'system', 'content': 'Você é uma inteligência artificial que irá resumir pedaços de texto de um livro sobre Processamento de Linguagem Natural, portanto, você deve capturar explicações e pontos importantes para o entendimento e mantendo uma linearidade no resumo.'}, {'role': 'user', 'content': '18\xa0 Tradução Automática\nAbordagens e Avaliação\n\n\n\n\n\nSheila Castilho \nHelena de Medeiros Caseli \n\n\n\nPublicado em:\n\n26/09/2023\n\n\n\n\nPDF\n\n18.1 Introdução\nA tradução automática (TA), também conhecida como tradução de máquina (em inglês, machine translation ou MT), refere-se à tradução de um texto eletrônico por um computador de uma língua para outra sem intervenção humana. Nesse sentido, convencionou-se chamar de língua (ou texto) fonte a língua de partida (origem) e língua (ou texto) alvo a língua de chegada (destino ou saída). Além de envolver a análise e interpretação (NLU) da língua-fonte e a geração (NLG) da língua-alvo, há a premissa fundamental de gerar uma s

In [ ]:
# Função para visualizar os resumos do Capítulo 8.
visualizar_resumo(resumo_Capitulo_8)

Resumo do trecho 0 :
A semântica estuda o significado das palavras e frases na linguagem. No entanto, a definição do significado de uma palavra é um problema complexo e depende da perspectiva teórica adotada. Existem duas
principais abordagens no estudo do significado: a representacionista/essencialista e a pragmática. A abordagem representacionista vê as palavras como substitutos de entidades extralinguísticas e busca
uma essência comum nos diferentes usos da palavra. Já a abordagem pragmática entende que o significado é variável e depende do contexto e da situação. No processamento de linguagem natural (PLN),
essas diferentes perspectivas se refletem em técnicas simbólicas e representações distribuídas. Além disso, existem os datasets com anotação semântica, que buscam criar uma representação estável entre
palavras e seus significados em contextos específicos. Essas abordagens têm impacto no desenvolvimento de recursos como wordnets e na anotação de significados em corpora.


Resumo 

In [ ]:
# Função para visualizar os resumos do Capítulo 18.
visualizar_resumo(resumo_Capitulo_18)

Resumo do trecho 0 :
A tradução automática (TA) é a tradução de um texto eletrônico de uma língua para outra sem intervenção humana. A TA evoluiu significativamente nos últimos anos com o avanço de modelos estatísticos e
neurais. Atualmente, é amplamente utilizada em diversas aplicações de comunicação. Existem diferentes abordagens para a TA, incluindo a tradução direta, baseada em regras, por interlíngua, baseada em
exemplos e estatística. A tradução direta mapeia palavras-fonte para palavras-alvo sem análise adicional, mas apresenta limitações. A tradução baseada em regras utiliza regras linguísticas para
realizar mapeamentos mais complexos, considerando a sintaxe das línguas envolvidas. A tradução por interlíngua utiliza uma língua intermediária para representar informações de outras línguas. A
tradução baseada em exemplos utiliza exemplos de traduções existentes para aprender a traduzir trechos de texto. A tradução estatística utiliza modelos estatísticos para extrair pares de trad

#### **Funções para a tarefa de Similaridade de Textos**

In [ ]:
# Definição da função para dividir um texto em chunks e obter embeddings utilizando OpenAIEmbeddings.
def splitter_OpenAIEmbeddings(text):
    # Configuração do text splitter para separar o texto por quebras de linha.
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=2500,
        chunk_overlap=200,
        length_function=len
    )

    # Divisão do texto em chunks.
    chunks = text_splitter.split_text(text)

    # Configuração e criação de embeddings utilizando OpenAIEmbeddings.
    embeddings = OpenAIEmbeddings(
        request_timeout=180,
        deployment='text-embedding-ada-002',
        chunk_size=10,
        openai_api_key=openai.api_key,
        show_progress_bar=True
    )

    return chunks, embeddings

In [ ]:
# Definição da função para buscar trechos similares na base de conhecimento do Capítulo 8.
def similarty_base_cap_8(query_user, k_user, chunks_8, embeddings_8, knowledge_base_8):
    # Obtém a representação vetorial da consulta do usuário.
    query = embeddings_8.embed_query(query_user)

    # Realiza a busca por similaridade na base de conhecimento do Capítulo 8.
    docs = knowledge_base_8.similarity_search_with_score_by_vector(query, k=k_user)

    # Lista para armazenar os resultados em formato JSON.
    json_data = []
    context = ''

    # Itera sobre os documentos retornados.
    for index, document in enumerate(docs):
        data = {
            "indice": index,
            "page_content": document[0].page_content,
            "score": float(document[1])
        }
        json_data.append(data)

    # Ordena os dados por score em ordem decrescente.
    sorted_data = sorted(json_data, key=lambda x: x["score"], reverse=True)

    print("\nOs trechos com a semelhança são os seguintes:\n\n")

    # Imprime os trechos similares ordenados por score.
    for i in sorted_data:
        print(f"O Trecho {i['indice']} com o score {i['score']} com o seguinte conteúdo:\n\n{i['page_content']}")
        print('\n\n')

In [ ]:
# Definição da função para buscar trechos similares na base de conhecimento do Capítulo 18.
def similarty_base_cap_18(query_user, k_user, chunks_18, embeddings_18, knowledge_base_18):
    # Obtém a representação vetorial da consulta do usuário.
    query = embeddings_18.embed_query(query_user)

    # Realiza a busca por similaridade na base de conhecimento do Capítulo 18.
    docs = knowledge_base_18.similarity_search_with_score_by_vector(query, k=k_user)

    # Lista para armazenar os resultados em formato JSON.
    json_data = []
    context = ''

    # Itera sobre os documentos retornados.
    for index, document in enumerate(docs):
        data = {
            "indice": index,
            "page_content": document[0].page_content,
            "score": float(document[1])
        }
        json_data.append(data)

    # Ordena os dados por score em ordem decrescente.
    sorted_data = sorted(json_data, key=lambda x: x["score"], reverse=True)

    print(f"\nOs trechos com a semelhança com o texto ({query_user}) são os seguintes:\n\n")

    # Imprime os trechos similares ordenados por score.
    for i in sorted_data:
        print(f"O Trecho {i['indice']} com o score {i['score']} com o seguinte conteúdo:\n\n{i['page_content']}")
        print('\n\n')

In [ ]:
# A chamada OpenAIEmbeddings.update_forward_refs() é usada para atualizar as referências internas da classe OpenAIEmbeddings.
# Isso pode ser útil em situações específicas, como resolver dependências cíclicas entre classes ou garantir que as referências estejam corretas após mudanças em tempo de execução.
# No entanto, é importante revisar a documentação ou contexto específico do código para entender completamente o motivo dessa chamada.
OpenAIEmbeddings.update_forward_refs()

#### **Criação de base de dados para a tarefa de Similaridade de Textos**

In [ ]:
# Obtendo chunks e embeddings para o Capítulo 8 utilizando a função splitter_OpenAIEmbeddings.
chunks_8, embeddings_8 = splitter_OpenAIEmbeddings(texto[0]['textoOffNormalize'])

# Criando a base de conhecimento FAISS a partir dos chunks e embeddings do Capítulo 8.
knowledge_base_8 = FAISS.from_texts(chunks_8, embeddings_8)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Obtendo chunks e embeddings para o Capítulo 18 utilizando a função splitter_OpenAIEmbeddings.
chunks_18, embeddings_18 = splitter_OpenAIEmbeddings(texto[1]['textoOffNormalize'])

# Criando a base de conhecimento FAISS a partir dos chunks e embeddings do Capítulo 18.
knowledge_base_18 = FAISS.from_texts(chunks_18, embeddings_18)

  0%|          | 0/5 [00:00<?, ?it/s]

#### **Aplicação da Tarefa de Similaridade de Textos**

In [ ]:
# Loop principal para interação contínua com o usuário.
while True:
    # Solicitação para digitar o número do capítulo desejado.
    entrada_usuario = int(input("Digite qual capítulo você quer recuperar informação: "))

    # Verifica se o número do capítulo é válido.
    if entrada_usuario == 8 or entrada_usuario == 18:
        # Solicitação para digitar a informação desejada.
        informacao = input("\nDigite qual informação quer recuperar: ")

        # Solicitação para informar a quantidade de trechos desejada.
        trechos = int(input("\nInforme a quantidade de informações que quer recuperar: "))

        # Verifica qual capítulo foi escolhido e chama a função correspondente.
        if entrada_usuario == 8:
            similarty_base_cap_8(informacao, trechos, chunks_8, embeddings_8, knowledge_base_8)
        elif entrada_usuario == 18:
            similarty_base_cap_18(informacao, trechos, chunks_18, embeddings_18, knowledge_base_18)

        # Solicitação para digitar 0 para sair ou qualquer outra tecla para continuar.
        sair_ou_ficar = int(input("\nDigite 0 para sair ou qualquer tecla para continuar: "))

        if sair_ou_ficar == 0:
            break
    else:
        print("\nVocê digitou um número de capítulo inválido na base de dados.")

Digite qual capitulo você quer recuperar informação: 1

Digitou um capitulo inválido na base de dados
Digite qual capitulo você quer recuperar informação: 8

Digite qual informação quer recuperar: o que é pnl

Informe quantidade de informações que quer recuperar: 3


  0%|          | 0/1 [00:00<?, ?it/s]


Os trechos com a semelhança são os seguintes:


O Trecho 2 com o score 0.6119176149368286 com o seguinte contéudo:

 Entre as técnicas simbólicas e as representações distribuídas existem ainda os datasets (ou corpora) com anotação semântica (veja Capítulo 14), uma terceira maneira de lidar com o significado no PLN. Se, por um lado, tais datasets se alinham às abordagens probabilísticas, uma vez que podem ser usados como fonte para o aprendizado de máquina (para o aprendizado do significado), por outro lado, a atividade de anotação de significado se alinha às abordagens representacionistas. Neste tipo de anotação (também chamada de anotação de word senses), cada palavra (ou segmento de texto) é anotada com informação relativa ao significado de acordo com o contexto específico em que aparece no corpus. A informação relativa ao significado, por sua vez, vem de fontes externas (como dicionários, wordnets, verbnets e framenets) e a tarefa de anotação pode ser descrita como um trabalho de d

  0%|          | 0/1 [00:00<?, ?it/s]


Os trechos com a semelhança com o texto(o que é pnl) são os seguintes:


O Trecho 4 com o score 0.6026187539100647 com o seguinte contéudo:

 A adoção de taxonomias de erros na ATA oferece diversos benefícios, tais como identificar tipos específicos de erros nas saídas de TA, fornecer relatórios detalhados de erros para o aprimoramento dos sistemas, e fornecer informações aos clientes sobre a qualidade da tradução. Além disso, provedores de serviços linguísticos utilizam taxonomias e avaliações de severidade29 para monitorar o trabalho de tradutores. A anotação de erro também ajuda a investigar as relações entre tipos específicos de erros e as preferências de usuários ou pós-editores, bem como avaliar o impacto de diferentes tipos de erros em várias etapas do processo de pós-edição.
Contudo, entre as principais limitações dessa estratégia, destaca-se que a anotação manual de erros é um processo caro e demorado, demandando um investimento significativo de tempo. Além disso, essa avalia

#### **Aplicação da Tarefa de Sistemas de Perguntas e Respostas em cima dos capitulos 8 e 18 selecionados**

In [ ]:
# Definição da função para dividir um texto em chunks e obter embeddings utilizando OpenAIEmbeddings.
def splitter_OpenAIEmbeddings(text):
    # Configuração do text splitter para separar o texto por quebras de linha.
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=2500,
        chunk_overlap=200,
        length_function=len
    )

    # Divisão do texto em chunks.
    chunks = text_splitter.split_text(text)

    # Configuração e criação de embeddings utilizando OpenAIEmbeddings.
    embeddings = OpenAIEmbeddings(
        request_timeout=180,
        deployment='text-embedding-ada-002',
        chunk_size=10,
        openai_api_key=openai.api_key,
        show_progress_bar=True
    )

    return chunks, embeddings

In [ ]:
from requests.models import Response

def requestOpenAI(query_user, chunks, embeddings, knowledge_base, mensagens):
    pergunta = query_user
    query = embeddings.embed_query(query_user)
    docs = knowledge_base.similarity_search_with_score_by_vector(query, k=4)
    json_data = []
    context = ''

    # Itera sobre os documentos retornados.
    for index, document in enumerate(docs):
        data = {
            "indice": index,
            "page_content": document[0].page_content,
            "score": float(document[1])
        }
        json_data.append(data)
        context += f'\nTrecho {index}:\n {document[0].page_content}'

    # Template para criar o prompt com o contexto e a pergunta.
    prompt_template = "Use as seguintes partes dos trechos para responder à pergunta. Contexto:\n{Context}.\n\nPergunta: {Pergunta}"
    prompt = prompt_template.replace("{Context}", context).replace("{Pergunta}", pergunta)

    # Adiciona a mensagem do usuário com o prompt ao histórico de mensagens.
    mensagens.append({'role': 'user', 'content': prompt})

    total_tokens = 0

    # Calcula o total de tokens nas mensagens para evitar ultrapassar o limite de 32.000 tokens.
    for mensagem in mensagens:
        tokens = mensagem['content']
        total_tokens += len(mensagem['content'])

    # Se ultrapassar o limite, remove algumas mensagens antigas.
    if total_tokens > 32000 and len(mensagens) > 1:
        del mensagens[1:3]

    # Chama a API da OpenAI para obter a resposta.
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k-0613",
        messages=mensagens,
        max_tokens=1024,
        temperature=0.4,
    )

    # Remove a última mensagem adicionada ao histórico antes de atualizar com a resposta.
    mensagens.pop()
    # Adiciona a pergunta do usuário e a resposta do modelo ao histórico de mensagens.
    mensagens.append({'role': 'user', 'content': pergunta})
    mensagens.append({'role': 'assistant', 'content': response["choices"][0]["message"]["content"]})

    return mensagens, response["choices"][0]["message"]["content"]

In [ ]:
# Concatenando os textos normalizados dos Capítulos 8 e 18 para criar o texto base.
texto_base = texto[0]['textoOffNormalize'] + texto[1]['textoOffNormalize']

In [ ]:
# Obtendo chunks e embeddings para o texto base utilizando a função splitter_OpenAIEmbeddings.
chunks, embeddings = splitter_OpenAIEmbeddings(texto_base)

# Criando a base de conhecimento FAISS a partir dos chunks e embeddings do texto base.
knowledge_base = FAISS.from_texts(chunks, embeddings)

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
print("Sou um chatBOT que irei de ajudar sobre dúvidas sobre os capítulos 8 e 18 e sobre o tema de Processamento de Linguagem Natural. Digite 'sair' caso queira encerrar a conversa.\n")

# Loop principal para interação contínua com o usuário.
while True:
    mensagens = []
    # Mensagem do sistema explicando o papel da inteligência artificial na conversa.
    mensagens.append({'role': 'system', 'content': "Você é uma inteligência artificial especialista sobre o tema de Processamento de Linguagem Natural. Você receberá trechos que podem ser pertinentes à pergunta do usuário, utilize-os somente para complementar a sua resposta, mas nunca utilize somente o contexto deles. Dê prioridade ao contexto da conversa."})

    # Solicitação para o usuário digitar sua mensagem.
    entrada_usuario = input("Você: ")

    # Verifica se o usuário deseja encerrar a conversa.
    if entrada_usuario.lower().strip() == "sair":
        break

    total_tokens = 0

    # Calcula o total de tokens nas mensagens para evitar ultrapassar o limite de 32.000 tokens.
    for mensagem in mensagens:
        tokens = mensagem['content']
        total_tokens += len(mensagem['content'])

    # Se ultrapassar o limite, remove algumas mensagens antigas.
    if total_tokens > 32000 and len(mensagens) > 1:
        del mensagens[1:3]

    # Chama a função para interagir com a OpenAI e obter a resposta do bot.
    mensagens, response_bot = requestOpenAI(entrada_usuario, chunks, embeddings, knowledge_base, mensagens)
    print("\nBot: " + response_bot + "\n")

Sou um chatBOT que irei de ajudar sobre dúvidas sobre os capitulos 8 e 18 e sobre o tema de Processamento de Linguagem Natural. Digite sair, caso queria sair da conversa

Você: O que é PNL ?


  0%|          | 0/1 [00:00<?, ?it/s]


Bot: PNL é a sigla para Processamento de Linguagem Natural. É uma área da inteligência artificial que se dedica ao estudo e desenvolvimento de técnicas e algoritmos para que os computadores possam compreender, interpretar e gerar linguagem humana de forma natural. O objetivo do PNL é permitir que os computadores possam interagir e comunicar-se com os seres humanos de maneira mais eficiente e eficaz, utilizando a linguagem como meio de comunicação.

Você: PLN é do campo de machine learning e ela engloba a tarefa de tradução de texto ?


  0%|          | 0/1 [00:00<?, ?it/s]


Bot: Sim, PLN (Processamento de Linguagem Natural) é uma área do campo de machine learning que engloba a tarefa de tradução de texto. O uso de modelos de tradução neural, que são baseados em redes neurais artificiais, é uma abordagem comum no PLN para realizar a tradução automática de uma sentença-fonte para uma sentença-alvo. Esses modelos têm sido amplamente utilizados e têm proporcionado melhorias significativas na qualidade e fluidez das traduções geradas. No entanto, a tradução neural também apresenta desafios, como a compreensão do contexto e nuances linguísticas, além de ser considerada uma caixa-preta, o que dificulta a interpretação dos processos internos do modelo.

Você: então fale mais sobre a tarefa de tradução


  0%|          | 0/1 [00:00<?, ?it/s]


Bot: A tarefa de tradução consiste em converter um texto escrito em um idioma de origem para um texto equivalente em um idioma de destino. É uma tarefa desafiadora, pois requer o entendimento do significado e da estrutura gramatical das frases em ambos os idiomas, bem como a capacidade de expressar essas informações de forma precisa e fluente.

Existem várias abordagens para a tradução automática, desde as mais simples, como a tradução direta palavra-a-palavra, até as mais avançadas, como a tradução neural baseada em redes neurais artificiais. A tradução direta envolve o mapeamento direto de palavras de um idioma para outro, sem passar por outros níveis de análise. Já a tradução neural utiliza modelos de aprendizado de máquina para aprender padrões e relações entre as palavras nos dois idiomas, permitindo uma tradução mais precisa e fluente.

Além disso, existem outras abordagens intermediárias, como a tradução baseada em regras, onde o conhecimento linguístico é explicitamente mapead